# Regular Genomics Project **RNA Localisation**


---

**Problem definition**:

TODO


### **1.** Data Preparation

Firstly, we import several necessary packages and load in our data:

In [5]:
import pandas as pd
import numpy as np
from notes.utils import read_model_file
from models.utils import plot_line_graph, box_plot
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
import datetime

In [6]:
# Initializing test/train split


colab = False  #### Set colab flag ####

if colab:
    np.random.seed(3)
    url = 'https://www.dropbox.com/s/hv4uau8q4wwg00k/final_data.csv?dl=1'
    data_org = pd.read_csv(url)
    test_data = data_org.sample(frac=0.1)
    train_data = data_org.drop(test_data.index)
else:
    np.random.seed(3)
    data_org = pd.read_csv('~/Downloads/final_data.csv')
    test_data = data_org.sample(frac=0.1)
    train_data = data_org.drop(test_data.index) # TODO: note: we also have to preprocess the test set similary
    # TODO: colab

data_org

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

### **2** |  Initializations

We use as baseline model the [RNATracker](https://github.com/HarveyYan/RNATracker/blob/master/Models/cnn_bilstm_attention.py) model and a CNN model.


In [ ]:
max_seq_len = train_data['seq'].apply(lambda x: len(x)).max()
print(max_seq_len)
# MODEL
model_path = "model_architectures/CNN_architecture4.yaml"

# Path where to save viz
model_architecture_path = "model_architecture_viz/simple_cnn.png"

params_dict = read_model_file(model_path, max_seq_len)
param_dataLoader_valid = params_dict['param_dataLoader_valid']
param_dataLoader_train = params_dict['param_dataLoader_train']
params_model = params_dict['params_model']
params_train = params_dict['params_train']


34526


In [ ]:
# training, only need for the model initialization to change in general
from models import CNN

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

# 80/20 split
train_split, valid_split = train_test_split(train_data, random_state=42, test_size=0.2)


# for i, (train_split, valid_split) in enumerate(folds):
model = CNN(
            input_size=(max_seq_len, 4),
            params_model=params_model,
checkpoint_filepath = "/checkpoints")

if checkpoint_filepath is None:
    checkpoint_filepath = '/checkpoint'

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    save_freq = "epoch")

history = model.fit_and_evaluate(train_data=train_split, eval_data=valid_split,
                                 callback = [model_checkpoint_callback]
                                 params_train_dataLoader=param_dataLoader_train,
                                 params_eval_dataLoader=param_dataLoader_valid,
                                 params_train=params_train)



# results = model.evaluate(eval_data=valid_split, **param_dataLoader_valid)
# results = dict(zip(model.model.metrics_names, results))

# VALIDATION_ACCURACY.append(results['accuracy'])
# VALIDATION_LOSS.append(results['loss'])

Epoch 1/40


2023-06-29 14:34:34.335092: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


 12/311 [>.............................] - ETA: 13:05 - loss: 124.8965 - accuracy: 0.1120 - kullback_leibler_divergence: 8.7094

KeyboardInterrupt: 

In [ ]:
model.print_model(model_architecture_path)
import datetime
import pydot
time_date = datetime.datetime.now().date()
# time_hour = datetime.datetime.now().time()

model_output = f"model_outputs/simple_cnn_{time_date}.h5"

model.save_model(model_output)

In [ ]:
a = model.summary()
print(a)


plt_data = [history.history['loss'], history.history['val_loss']]
plot_line_graph(plt_data, "Loss Graph", 'loss', 'epoch', ['train', 'val'])

plt_data = [history.history['accuracy'], history.history['val_accuracy']]
plot_line_graph(plt_data, "Accuracy Graph", 'accuracy', 'epoch', ['train', 'val'])

plt_data = [history.history['kullback_leibler_divergence'], history.history['val_kullback_leibler_divergence']]
plot_line_graph(plt_data, "kullback_leibler_divergence", 'kullback_leibler_divergence', 'epoch', ['train', 'val'])

plt_data = [history.history['tf_pearson'], history.history['val_tf_pearson']]
plot_line_graph(plt_data, "tf_pearson_Graph", 'tf_pearson', 'epoch', ['train', 'val'])

box_plot(train_data)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 34526, 4)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 34526, 16)    720         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 34526, 16)   64          ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 conv1d_1 (Conv1D)              (None, 34526, 16)    2832        ['batch_normalization[0][0]']

NameError: name 'history' is not defined

In [ ]:
test_result = model.evaluate(test_data, **param_dataLoader_valid)
result = dict(zip(model.model.metrics_names, test_result))
TEST_ACCURACY = result['accuracy']
TEST_LOSS = result['loss']

In [ ]:
VALIDATION_ACCURACY

In [ ]:
VALIDATION_LOSS